# Introduction to RL Agent

This notebook introduces the basics of reinforcement learning agents.

## Setup

Install required dependencies for the RL environment.

## Import Libraries

In [1]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

## Create Environment

In [3]:
# Create the LunarLander-v3 environment
env = gym.make("LunarLander-v3")
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)

Observation space: Box([ -2.5        -2.5       -10.        -10.         -6.2831855 -10.
  -0.         -0.       ], [ 2.5        2.5       10.        10.         6.2831855 10.
  1.         1.       ], (8,), float32)
Action space: Discrete(4)


## Define the PPO Model

In [36]:
# Create the PPO model
model = PPO(
    "MlpPolicy",
    env,
    learning_rate=3e-4,          # Good default for PPO
    n_steps=2048,                 # Steps per env before update (higher = more data per update)
    batch_size=64,                # Minibatch size for optimization
    n_epochs=10,                  # Times to iterate through batch
    gamma=0.99,                   # Discount factor (how much to value future rewards)
    gae_lambda=0.95,              # Advantage estimation parameter
    clip_range=0.2,               # PPO clipping parameter
    ent_coef=0.01,                # Entropy bonus (encourages exploration)
    verbose=1,
    device='cuda',
)

print("Model created successfully!")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Model created successfully!


## Train the Model

In [37]:
# Train the agent for 1 million timesteps
total_timesteps = 1e6
model.learn(total_timesteps=total_timesteps)

# Save the trained model
model.save("ppo_lunarlander")
print("Training complete! Model saved.")

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 95.4     |
|    ep_rew_mean     | -165     |
| time/              |          |
|    fps             | 850      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 92.1       |
|    ep_rew_mean          | -169       |
| time/                   |            |
|    fps                  | 708        |
|    iterations           | 2          |
|    time_elapsed         | 5          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01023638 |
|    clip_fraction        | 0.0587     |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.38      |
|    explained_variance   | 0.00195    |
|    learning_rate        | 0.0003     |
|   

## Evaluate the Trained Model

In [34]:
# Evaluate the trained agent
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, deterministic=True)

print(f"Mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")

Mean reward: 261.54 +/- 16.43


## Visualize the Results

In [38]:
# Record a video of the trained agent
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(env_id, model, video_length=500, prefix='ppo-lunarlander-success'):
    """
    Record a video of the agent
    """
    eval_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])
    
    # Start recording
    eval_env = VecVideoRecorder(eval_env, f"videos/{prefix}",
                               record_video_trigger=lambda x: x == 0, video_length=video_length,
                               name_prefix=prefix)
    
    obs = eval_env.reset()
    for _ in range(video_length):
        action, _ = model.predict(obs, deterministic=True)
        obs, _, _, _ = eval_env.step(action)
    
    # Close the video recorder
    eval_env.close()

record_video('LunarLander-v3', model, video_length=1000)
print("Video recorded successfully!")

Saving video to /home/harris/dev/huggingface_rl/tutorials/videos/ppo-lunarlander-success/ppo-lunarlander-success-step-0-to-step-1000.mp4
MoviePy - Building video /home/harris/dev/huggingface_rl/tutorials/videos/ppo-lunarlander-success/ppo-lunarlander-success-step-0-to-step-1000.mp4.
MoviePy - Writing video /home/harris/dev/huggingface_rl/tutorials/videos/ppo-lunarlander-success/ppo-lunarlander-success-step-0-to-step-1000.mp4



MoviePy - Done !
MoviePy - video ready /home/harris/dev/huggingface_rl/tutorials/videos/ppo-lunarlander-success/ppo-lunarlander-success-step-0-to-step-1000.mp4
Video recorded successfully!
